# Semantic Router: 의미 기반 자동 라우팅

이 노트북에서는 **질문의 의미를 분석**하여 **가장 적합한 전문가 프롬프트로 자동 연결**하는 **Semantic Router** 기법을 배웁니다.

## 이전 Router와의 차이점

```
┌────────────────────────────────────────────────────────────────────┐
│            논리적 라우터 vs 의미론적 라우터                         │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   논리적 라우터 (이전 노트북):                                      │
│   • LLM이 직접 판단                                                │
│   • "Python 질문이니까 python_docs로!"                             │
│   • 장점: 복잡한 판단 가능                                         │
│   • 단점: LLM 호출 필요 (비용, 시간)                               │
│                                                                    │
│   의미론적 라우터 (이 노트북):                                      │
│   • 임베딩 유사도로 자동 판단                                      │
│   • 질문 임베딩 ↔ 프롬프트 임베딩 비교                              │
│   • 장점: 빠르고 저렴                                              │
│   • 단점: 복잡한 조건 처리 어려움                                   │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## Semantic Router의 원리

```
┌────────────────────────────────────────────────────────────────────┐
│                  Semantic Router 동작 원리                         │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   [준비 단계]                                                      │
│   물리학 프롬프트 ───▶ 임베딩 [0.12, 0.45, ...]                    │
│   수학 프롬프트   ───▶ 임베딩 [0.34, -0.21, ...]                   │
│                                                                    │
│   [실행 단계]                                                      │
│   질문: "블랙홀이란 무엇인가요?"                                   │
│          │                                                        │
│          ▼                                                        │
│   질문 임베딩 [0.11, 0.43, ...]                                    │
│          │                                                        │
│          ▼                                                        │
│   ┌─────────────────────────────────────────┐                      │
│   │          코사인 유사도 비교              │                      │
│   │  물리학 프롬프트: 0.95 ← 가장 높음! ✅   │                      │
│   │  수학 프롬프트:   0.42                   │                      │
│   └─────────────────────────────────────────┘                      │
│          │                                                        │
│          ▼                                                        │
│   물리학 프롬프트 선택!                                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama numpy

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 전문가 프롬프트 정의

각 분야의 전문가 역할을 하는 프롬프트를 정의합니다.

In [ ]:
# 물리학 전문가 프롬프트
physics_template = '''당신은 매우 똑똑한 물리학 교수입니다. 
당신은 물리학에 대한 질문에 간결하고 쉽게 이해할 수 있는 방식으로 대답하는 데 뛰어납니다.
당신이 질문에 대한 답을 모를 때는 모른다고 인정합니다.
다음 질문에 답하세요: {query}
'''

# 수학 전문가 프롬프트  
math_template = '''
당신은 매우 뛰어난 수학자입니다. 당신은 수학 문제에 답하는 데 뛰어납니다.
당신은 어려운 문제를 구성 요소로 분해하고 구성 요소를 해결한 다음
함께 모아 더 넓은 질문에 대답합니다.
다음 질문에 답하세요: {query}
'''

prompt_templates = [physics_template, math_template]

print("✅ 전문가 프롬프트 정의 완료")
print("   - 물리학 전문가")
print("   - 수학 전문가")

# 3. 프롬프트 임베딩 생성

각 프롬프트를 벡터로 변환합니다. 이 벡터들이 라우팅의 기준이 됩니다.

In [ ]:
from langchain_ollama import OllamaEmbeddings

# 임베딩 모델
embeddings = OllamaEmbeddings(model='nomic-embed-text')

# 프롬프트들을 임베딩
prompt_embeddings = embeddings.embed_documents(prompt_templates)

print("✅ 프롬프트 임베딩 완료")
print(f"   - 물리학 프롬프트 벡터: [{prompt_embeddings[0][0]:.4f}, {prompt_embeddings[0][1]:.4f}, ...]")
print(f"   - 수학 프롬프트 벡터: [{prompt_embeddings[1][0]:.4f}, {prompt_embeddings[1][1]:.4f}, ...]")
print(f"   - 벡터 차원: {len(prompt_embeddings[0])}")

# 4. 코사인 유사도 함수

## 코사인 유사도란?

두 벡터가 **얼마나 같은 방향을 가리키는지** 측정하는 방법입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                   코사인 유사도 쉬운 설명                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   값의 범위: -1 ~ 1                                                │
│                                                                    │
│   1.0  : 완전히 같은 방향 (매우 유사) ✅                           │
│   0.0  : 수직 (관련 없음)                                          │
│   -1.0 : 완전히 반대 방향 (반대)                                   │
│                                                                    │
│   예시:                                                            │
│   "블랙홀" 임베딩 ←→ 물리학 프롬프트 임베딩 = 0.95 (매우 유사!)    │
│   "블랙홀" 임베딩 ←→ 수학 프롬프트 임베딩   = 0.42 (덜 유사)       │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain.utils.math import cosine_similarity

# 테스트: 두 벡터의 유사도 계산
test_query = "블랙홀이란 무엇인가요?"
query_embedding = embeddings.embed_query(test_query)

# 각 프롬프트와의 유사도 계산
similarities = cosine_similarity([query_embedding], prompt_embeddings)[0]

print(f"질문: {test_query}\n")
print("=== 각 프롬프트와의 유사도 ===")
print(f"물리학 프롬프트: {similarities[0]:.4f}")
print(f"수학 프롬프트:   {similarities[1]:.4f}")
print(f"\n→ 물리학 프롬프트가 더 유사하므로 선택됨!")

# 5. Semantic Router 구현

In [ ]:
from langchain_core.runnables import chain
from langchain_core.prompts import PromptTemplate

@chain
def prompt_router(query):
    """
    Semantic Router: 질문의 의미를 분석하여 적절한 프롬프트 선택
    
    1. 질문을 임베딩으로 변환
    2. 각 프롬프트 임베딩과 유사도 계산
    3. 가장 유사한 프롬프트 반환
    """
    # 질문 임베딩 생성
    query_embedding = embeddings.embed_query(query)
    
    # 각 프롬프트와의 유사도 계산
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    
    # 가장 유사한 프롬프트 선택
    most_similar_idx = similarity.argmax()
    most_similar = prompt_templates[most_similar_idx]
    
    # 어떤 프롬프트가 선택되었는지 출력
    if most_similar == math_template:
        print('📐 수학 프롬프트 사용')
    else:
        print('🔭 물리 프롬프트 사용')
    
    return PromptTemplate.from_template(most_similar)

print("✅ Semantic Router 준비 완료")

# 6. 전체 Semantic Router 체인 구축

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOllama(model='llama3.2', temperature=0)

# Semantic Router 체인
# 1. prompt_router: 질문에 맞는 프롬프트 선택
# 2. ChatOllama: LLM으로 답변 생성
# 3. StrOutputParser: 문자열로 출력
semantic_router = prompt_router | llm | StrOutputParser()

print("✅ Semantic Router 체인 준비 완료")

In [ ]:
# 물리학 질문 테스트
physics_question = '블랙홀이란 무엇인가요?'

print("=== 물리학 질문 테스트 ===")
print(f"질문: {physics_question}\n")

result = semantic_router.invoke(physics_question)
print(f"\n답변: {result}")

In [ ]:
# 수학 질문 테스트
math_question = '2차 방정식의 근의 공식을 알려주세요'

print("=== 수학 질문 테스트 ===")
print(f"질문: {math_question}\n")

result = semantic_router.invoke(math_question)
print(f"\n답변: {result}")

# 7. 다양한 질문으로 테스트

In [ ]:
test_questions = [
    "빛의 속도는 얼마인가요?",
    "피타고라스 정리를 설명해주세요",
    "상대성 이론이란 무엇인가요?",
    "미분과 적분의 관계는?",
    "중력은 어떻게 작용하나요?",
    "행렬의 곱셈은 어떻게 하나요?",
]

print("=== 다양한 질문 라우팅 테스트 ===")
for q in test_questions:
    print(f"\n{'='*50}")
    print(f"질문: {q}")
    
    # 유사도 확인
    query_emb = embeddings.embed_query(q)
    sims = cosine_similarity([query_emb], prompt_embeddings)[0]
    print(f"유사도 - 물리: {sims[0]:.3f}, 수학: {sims[1]:.3f}")
    
    # 라우팅 결과
    result = semantic_router.invoke(q)
    print(f"답변: {result[:100]}...")

---

## 정리: Semantic Router

### 논리적 라우터 vs 의미론적 라우터 비교

| 항목 | 논리적 라우터 | 의미론적 라우터 |
|------|--------------|----------------|
| **판단 주체** | LLM | 임베딩 유사도 |
| **속도** | 느림 (LLM 호출) | 빠름 |
| **비용** | LLM 비용 발생 | 임베딩만 필요 |
| **복잡한 조건** | 가능 | 어려움 |
| **확장성** | 프롬프트 수정 필요 | 템플릿 추가만 |

### 핵심 코드

```python
# 1. 프롬프트 임베딩 생성
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# 2. Semantic Router 함수
@chain
def prompt_router(query):
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    return PromptTemplate.from_template(most_similar)

# 3. 체인 구성
semantic_router = prompt_router | llm | StrOutputParser()
```

### 사용 사례

| 시나리오 | 라우팅 대상 |
|----------|------------|
| 학과별 질문 | 물리학/수학/화학/생물학 전문가 |
| 톤/스타일 | 공식적/친근한/유머러스한 답변 |
| 언어 수준 | 전문가용/초보자용 설명 |
| 응답 형식 | 상세 설명/요약/리스트 형태 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI()

# 변경
from langchain_ollama import ChatOllama, OllamaEmbeddings
embeddings = OllamaEmbeddings(model='nomic-embed-text')
llm = ChatOllama(model='llama3.2')
```

## 다음 단계

**메타데이터를 활용한 필터링 검색**인 **Self-Query Retriever**를 배웁니다. (22번 노트북)